## 前言

我们都知道 python 中可以是 `threading` 模块实现多线程, 但是模块并没有提供暂停, 恢复和停止线程的方法, 一旦线程对象调用 `start` 方法后, 只能等到对应的方法函数运行完毕. 也就是说一旦 `start` 后, 线程就属于失控状态. 对于函数中没有循环，可以使用`join()`来结束循环。

不过, 我们可以自己实现这些. 一般的方法就是循环地判断一个标志位, 一旦标志位到达到预定的值, 就退出循环. 这样就能做到退出线程了

## 方法一：Event

- 文档说明：Event.wait(timeout=None)

```

    Block until the internal flag is true. If the internal flag is true on entry, return immediately. Otherwise, block until another thread calls set() to set the flag to true, or until the optional timeout occurs.

    阻塞, 直到内部的标志位为True时. 如果在内部的标志位在进入时为True时, 立即返回. 否则, 阻塞直到其他线程调用`set()`方法将标志位设为True, 或者到达了可选参数的timeout时间.


    When the timeout argument is present and not None, it should be a floating point number specifying a timeout for the operation in seconds (or fractions thereof).

    This method returns the internal flag on exit, so it will always return True except if a timeout is given and the operation times out.

    当给定了timeout参数且不为None, 它应该是以秒为单位的浮点数，指定操作的超时时间（或是分数）。

    此方法在退出时返回内部标志，因此除非给定了超时参数且操作确实超时了，否则它将始终返回True。


    Changed in version 2.7: Previously, the method always returned None.

    2.7版本以前, 这个方法总会返回None.
    
```

### 实现原理

利用`wait` 的阻塞机制, 就能够实现暂停和恢复了, 再配合循环判断标识位, 就能实现退出了, 下面是代码示例:


In [1]:
# !/usr/bin/env python
# coding: utf-8

import threading
import time


class Job(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(Job, self).__init__(*args, **kwargs)
        self.__flag = threading.Event()  # 用于暂停线程的标识
        self.__flag.set()  # 设置为True
        self.__running = threading.Event()  # 用于停止线程的标识
        self.__running.set()  # 将running设置为True

    def run(self):
        while self.__running.isSet():
            self.__flag.wait()  # 为True时立即返回, 为False时阻塞直到内部的标识位为True后返回
            print(int(time.time()))
            time.sleep(1)

    def pause(self):
        self.__flag.clear()  # 设置为False, 让线程阻塞

    def resume(self):
        self.__flag.set()  # 设置为True, 让线程停止阻塞

    def stop(self):
        self.__running.clear()  # 设置为 False
        self.__flag.set()  # 将线程从暂停状态恢复, 如果已经暂停的话

In [2]:
# 下面是测试代码:
a = Job()
a.start()
time.sleep(3)
a.pause()
time.sleep(3)
a.resume()
time.sleep(3)
a.pause()
time.sleep(2)
a.stop()

1590401672
1590401673
1590401674
1590401678
1590401679
1590401680
1590401683


### 说明

这完成了暂停, 恢复和停止的功能. 但是这里有一个缺点:   
无论是暂停还是停止, 都**不是瞬时的**, 必须等待 `run` 函数内部的运行到达标志位判断时才有效. 也就是说操作会滞后一次.

但是这有时也不一定是坏事. 如果 `run` 函数中涉及了文件操作或数据库操作等, 完整地运行一次后再退出, 反而能够执行剩余的资源释放操作的代码 (例如各种 close). 不会出现程序的文件操作符超出上限, 数据库连接未释放等尴尬的情况.

## 方法二：信号量Sempaphore

提供`acquire`方法和`release`方法，每当调用`acquire`方法的时候，如果内部计数器大于0，则将其减1，如果内部计数器等于0，则会阻塞该线程，知道有线程调用了`release`方法将内部计数器更新到大于1位置。

```py
class threading.Sempaphore(value = 1)
```
semaphore 是一个内部的计数器,它会随着`acquire()`的请求减小,也会随着`release()`的释放增加。这个计数器的值不会小于零,当`acquier()` 发现计数器的值为0时,那么当前对象会等待直到其他对象`release()`为止。

实现参考上一种方法：
- acquier(blocking = True ,timeout = None)  
代替 `Event.clear()` 方法
- release()  
代替 `Event.set()` 方法

### 说明

在高并发环境中反复调用 `acquier()`和 `release()` 会产生的锁资源开销比上一种方法大，不如上一种方法好用。

## 强行终止线程

根据线程 id 强行终止线程

In [3]:
import threading
import time
import inspect
import ctypes


def _async_raise(tid: int, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("Invalid thread id")
    elif res != 1:
        # if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")


def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)


def test():
    while True:
        print('-------')
        time.sleep(1)


if __name__ == "__main__":
    t = threading.Thread(target=test)
    t.start()
    time.sleep(5.2)
    print("main thread sleep finish")
    stop_thread(t)

-------
-------
-------
-------
-------
-------
main thread sleep finish
